In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDAF")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:38:32 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:38:32 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:38:32 INFO SparkContext: Java version 11.0.26
25/03/13 16:38:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".
25/03/13 16:38:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/13 16:38:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


spark: SparkSession = org.apache.spark.sql.SparkSession@166ae44a
import spark.implicits._

## Простой пример: Среднее

In [4]:
val data1 = Seq(
                ("Michael", 1, 3000),
                ("Michael", 2, 6000),
                ("Andy", 1, 4500),
                ("Andy", 2, 2500),
                ("Justin", 3, 3500),
                ("Berta", 3, 4000))
            .toDF("name", "depId", "salary")

data1.show()

+-------+-----+------+
|   name|depId|salary|
+-------+-----+------+
|Michael|    1|  3000|
|Michael|    2|  6000|
|   Andy|    1|  4500|
|   Andy|    2|  2500|
| Justin|    3|  3500|
|  Berta|    3|  4000|
+-------+-----+------+



data1: DataFrame = [name: string, depId: int ... 1 more field]

In [5]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Average(var sum: Long, var count: Long)

class MyAverage extends Aggregator[Long, Average, Double] {
  // Начальное значение. Должно соответствовать свойству: любое b + zero = b
  def zero: Average = Average(0L, 0L)
  // Объединение двух значений в новое значение.
  // Для повышения производительности функция может изменять `buffer` и 
  // возвращать его вместо создания нового объекта.
  def reduce(buffer: Average, data: Long): Average = {
    buffer.sum += data
    buffer.count += 1
    buffer
  }
  // Объединение двух промежуточных значения
  def merge(b1: Average, b2: Average): Average = {
    b1.sum += b2.sum
    b1.count += b2.count
    b1
  }
  // Преобразование выходных данных
  def finish(reduction: Average): Double = reduction.sum.toDouble / reduction.count
  // Кодировщик для типа промежуточного значения
  def bufferEncoder: Encoder[Average] = Encoders.product
  // Кодировщик для типа выходного значения
  def outputEncoder: Encoder[Double] = Encoders.scalaDouble
}

defined class Average
defined class MyAverage

In [6]:
val myAverageUDAF = udaf(new MyAverage)

myAverageUDAF: UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cmd5$Helper$MyAverage@5b22064b,
  ExpressionEncoder(
    BoundReference(0, LongType, false),
    AssertNotNull(
      UpCast(
        GetColumnByOrdinal(0, LongType),
        LongType,
        List("- root class: \"long\"")
      ),
      List("- root class: \"long\"")
    ),
    Long
  ),
  None,
  true,
  true
)

In [7]:
data1
    .groupBy("depId")
    .agg(myAverageUDAF($"salary").as("average_salary"))
    .show()

+-----+--------------+
|depId|average_salary|
+-----+--------------+
|    1|        3750.0|
|    2|        4250.0|
|    3|        3750.0|
+-----+--------------+



In [8]:
data1
    .groupBy($"name")
    .agg(myAverageUDAF($"salary").as("average_salary"))
    .show

+-------+--------------+
|   name|average_salary|
+-------+--------------+
|Michael|        4500.0|
|   Andy|        3500.0|
| Justin|        3500.0|
|  Berta|        4000.0|
+-------+--------------+



## Продвинутый пример: сумма с ограничением

Каждый клиент получает один балл за каждую заказанную единицу товара, но неболее трёх баллов в одном заказе.

In [9]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [10]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Buffer(var value: Int) 

class PointAttribution extends Aggregator[Int, Buffer, Int] {
  val MAX_POINT_PER_ORDER = 3
  // Начальное значение. Должно соответствовать свойству: любое b + zero = b
  def zero: Buffer = Buffer(0)
  // Объединение двух значений в новое значение.
  // Для повышения производительности функция может изменять `buffer` и 
  // возвращать его вместо создания нового объекта.
  def reduce(buffer: Buffer, data: Int): Buffer = {
    val outputValue = if (data < MAX_POINT_PER_ORDER) data else MAX_POINT_PER_ORDER
    buffer.value += outputValue
    buffer
  }
  // Объединение двух промежуточных значения
  def merge(b1: Buffer, b2: Buffer): Buffer = {
    b1.value += b2.value
    b1
  }
  // Преобразование выходных данных
  def finish(reduction: Buffer): Int = reduction.value
  // Кодировщик для типа промежуточного значения
  def bufferEncoder: Encoder[Buffer] = Encoders.product
  // Кодировщик для типа выходного значения
  def outputEncoder: Encoder[Int] = Encoders.scalaInt
}

defined class Buffer
defined class PointAttribution

In [11]:
val pointAttribution = new PointAttribution
val pointAttributionUDAF = udaf(pointAttribution)

pointAttribution: PointAttribution = ammonite.$sess.cmd10$Helper$PointAttribution@668d50ee
pointAttributionUDAF: UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cmd10$Helper$PointAttribution@668d50ee,
  ExpressionEncoder(
    BoundReference(0, IntegerType, false),
    AssertNotNull(
      UpCast(
        GetColumnByOrdinal(0, IntegerType),
        IntegerType,
        List("- root class: \"int\"")
      ),
      List("- root class: \"int\"")
    ),
    Int
  ),
  None,
  true,
  true
)

In [ ]:
val t1 = System.nanoTime

data
  .groupBy($"firstName", $"lastName", $"state")
  .agg(sum("quantity"), pointAttributionUDAF($"quantity").as("point"))
  .show()

val duration1 = (System.nanoTime - t1) / 1e9d

+------------+--------+-----+-------------+-----+
|   firstName|lastName|state|sum(quantity)|point|
+------------+--------+-----+-------------+-----+
|Jean-Georges|  Perrin|   NC|            3|    3|
|Jean-Georges|  Perrin|   CA|            4|    3|
|      Holden|   Karau|   CA|           10|    6|
|       Ginni| Rometty|   NY|            7|    3|
+------------+--------+-----+-------------+-----+



t1: Long = 22905627996458L
duration1: Double = 2.065082583

In [13]:
println(s"duration1 = $duration1")

duration1 = 2.065082583


Вариант решения этой задачи без UDAF.

In [ ]:
val max = pointAttribution.MAX_POINT_PER_ORDER

val t2 = System.nanoTime
data
  .withColumn("point", when($"quantity".$greater(max), max).otherwise($"quantity"))
  .groupBy($"firstName", $"lastName", $"state")
  .agg(sum("quantity"), sum("point").as("point"))
  .show()
        
val duration2 = (System.nanoTime - t2) / 1e9d

+------------+--------+-----+-------------+-----+
|   firstName|lastName|state|sum(quantity)|point|
+------------+--------+-----+-------------+-----+
|Jean-Georges|  Perrin|   NC|            3|    3|
|Jean-Georges|  Perrin|   CA|            4|    3|
|      Holden|   Karau|   CA|           10|    6|
|       Ginni| Rometty|   NY|            7|    3|
+------------+--------+-----+-------------+-----+



max: Int = 3
t2: Long = 22911589203250L
duration2: Double = 3.464453083

In [15]:
println(s"duration1 = $duration1\nduration2 = $duration2\nduration1 - duration2 = ${duration1 - duration2}")

duration1 = 2.065082583
duration2 = 3.464453083
duration1 - duration2 = -1.3993704999999999
